In [1]:
import findspark
findspark = findspark.init()

import os
import datetime
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql import Row
import pyspark.sql.types as T


/Users/aravind_jarpala/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/aravind_jarpala/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [ ]:

spark = SparkSession.builder. \
        master("local[*]"). \
        appName('FileFormatComparison'). \
        config('spark.driver.host', 'localhost'). \
        config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.1"). \
        getOrCreate()

26/01/12 08:25:01 WARN Utils: Your hostname, Aravinds-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.102 instead (on interface en0)
26/01/12 08:25:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/aravind_jarpala/.ivy2/cache
The jars for the packages stored in: /Users/aravind_jarpala/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8fb7cf28-95f3-44d5-a138-a35b16576407;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/aravind_jarpala/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-avro_2.12;3.5.1 in central
	found org.tukaani#xz;1.9 in central
:: resolution report :: resolve 289ms :: artifacts dl 5ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.5.1 from central in [default]
	org.tukaani#xz;1.9 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-8fb7cf28-95f3-44d5-a138-a35b16576407
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/15ms)
26/01/12 08:25:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes whe

In [3]:
num_rows = 10000000

df = spark.range(0, num_rows)

for i in range(1, 10):
    if i % 2 == 0:
        df = df.withColumn(f'int_col_{i}', (rand() * 100).cast(T.IntegerType()))
    else:
        df = df.withColumn(f'string_col_{i}', (rand() * num_rows).cast(T.IntegerType()).cast('string'))

df.count()

10000000

In [4]:
df.show(5, truncate=False)

+---+------------+---------+------------+---------+------------+---------+------------+---------+------------+
|id |string_col_1|int_col_2|string_col_3|int_col_4|string_col_5|int_col_6|string_col_7|int_col_8|string_col_9|
+---+------------+---------+------------+---------+------------+---------+------------+---------+------------+
|0  |8161333     |75       |1013209     |86       |1047859     |29       |7227234     |1        |7110999     |
|1  |3935537     |56       |7066891     |34       |7579803     |19       |6073898     |49       |1365582     |
|2  |4425550     |36       |469843      |39       |6903911     |51       |6046763     |24       |2890999     |
|3  |4880694     |79       |9629211     |89       |8755482     |20       |6347102     |48       |3874642     |
|4  |1161286     |49       |2265589     |86       |8863354     |1        |2343175     |32       |742965      |
+---+------------+---------+------------+---------+------------+---------+------------+---------+------------+
o

In [ ]:
base_path = f"file-formats/tmp"

#write as different file formats
df.write.mode("overwrite").option("header", "true").csv(f"{base_path}/csv")
df.write.mode("overwrite").parquet(f"{base_path}/parquet")
df.write.mode("overwrite").format("avro").save(f"{base_path}/avro")

26/01/12 08:25:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
26/01/12 08:25:16 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


##### File Format Size

In [7]:
!du -sh file-formats/tmp/*

460M	file-formats/tmp/avro
648M	file-formats/tmp/csv
420M	file-formats/tmp/parquet


#### Columnar Pruning for Parquet

In [8]:
start_time_all = time.time()
spark.read.parquet(f"{base_path}/parquet").count()
end_time_all = time.time()
print(f"Parquet read time (All): {end_time_all - start_time_all:.2f} seconds")

Parquet read time (All): 0.38 seconds


In [9]:
print("\nTesting: Select Only 1 Column (Pruning)")
start_prune = time.time()
# Spark will literally skip the unnecessary columns when reading Parquet
spark.read.parquet(f"{base_path}/parquet").select("id").explain(True)
print(f"Time taken (Pruned): {time.time() - start_prune:.2f} seconds")


Testing: Select Only 1 Column (Pruning)
== Parsed Logical Plan ==
'Project ['id]
+- Relation [id#186L,string_col_1#187,int_col_2#188,string_col_3#189,int_col_4#190,string_col_5#191,int_col_6#192,string_col_7#193,int_col_8#194,string_col_9#195] parquet

== Analyzed Logical Plan ==
id: bigint
Project [id#186L]
+- Relation [id#186L,string_col_1#187,int_col_2#188,string_col_3#189,int_col_4#190,string_col_5#191,int_col_6#192,string_col_7#193,int_col_8#194,string_col_9#195] parquet

== Optimized Logical Plan ==
Project [id#186L]
+- Relation [id#186L,string_col_1#187,int_col_2#188,string_col_3#189,int_col_4#190,string_col_5#191,int_col_6#192,string_col_7#193,int_col_8#194,string_col_9#195] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [id#186L] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/aravind_jarpala/Downloads/Data Engineering/lab/data-engine..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:bigin

##### Columnar Pruning for CSV / AVRO

In [10]:
start_time_all = time.time()
spark.read.format("avro").load(f"{base_path}/avro").select("id").explain(True)
end_time_all = time.time()
print(f"Avro read time (All): {end_time_all - start_time_all:.2f} seconds")

== Parsed Logical Plan ==
'Project ['id]
+- Relation [id#208L,string_col_1#209,int_col_2#210,string_col_3#211,int_col_4#212,string_col_5#213,int_col_6#214,string_col_7#215,int_col_8#216,string_col_9#217] avro

== Analyzed Logical Plan ==
id: bigint
Project [id#208L]
+- Relation [id#208L,string_col_1#209,int_col_2#210,string_col_3#211,int_col_4#212,string_col_5#213,int_col_6#214,string_col_7#215,int_col_8#216,string_col_9#217] avro

== Optimized Logical Plan ==
Project [id#208L]
+- Relation [id#208L,string_col_1#209,int_col_2#210,string_col_3#211,int_col_4#212,string_col_5#213,int_col_6#214,string_col_7#215,int_col_8#216,string_col_9#217] avro

== Physical Plan ==
FileScan avro [id#208L] Batched: false, DataFilters: [], Format: Avro, Location: InMemoryFileIndex(1 paths)[file:/Users/aravind_jarpala/Downloads/Data Engineering/lab/data-engine..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:bigint>

Avro read time (All): 0.08 seconds


##### Schema Evolution

In [ ]:
# Add a new column and write as new version
df_v2 = spark.read.parquet(f"{base_path}/parquet"). \
    withColumn("new_col", lit("v2_data")). \
    coalesce(4)

In [19]:
df_v2.write.mode("append").parquet(f"{base_path}/parquet")

df_v2.write.format("avro"). \
    mode("append"). \
    save(f"{base_path}/avro")

In [29]:
spark.read.option("mergeSchema", "true"). \
    parquet(f"{base_path}/parquet"). \
    printSchema()

# parquet_v2_df.show(5, truncate=False)

root
 |-- id: long (nullable = true)
 |-- string_col_1: string (nullable = true)
 |-- int_col_2: integer (nullable = true)
 |-- string_col_3: string (nullable = true)
 |-- int_col_4: integer (nullable = true)
 |-- string_col_5: string (nullable = true)
 |-- int_col_6: integer (nullable = true)
 |-- string_col_7: string (nullable = true)
 |-- int_col_8: integer (nullable = true)
 |-- string_col_9: string (nullable = true)
 |-- new_col: string (nullable = true)

